In [41]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

import keras as K
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Dense, Input, TimeDistributed, Activation
from keras.optimizers import RMSprop, SGD, Adadelta, Adagrad
from keras.initializers import RandomUniform
np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utils.reader import ptb_raw_data
from utils.batcher import generate_epoch

In [38]:
EMBEDDING_SIZE=64
HIDDEN_SIZE=64
BATCH_SIZE=64
NUM_STEPS=8
NUM_EPOCHS_INIT_LR=4
NUM_EPOCHS_TOTAL=12
INITIAL_LR=20
LR_DECAY_RATE=0.75
MAX_NORM=5

In [4]:
def keras_generate_epoch(data, bs, ns):
    for x, y in generate_epoch(data, bs, ns):
        yield x, np.expand_dims(y, -1)
        

In [5]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [54]:
optimizer = RMSprop(lr=0.01, rho=0.999)
initializer = RandomUniform(minval=-0.15, maxval=0.15)

model = Sequential()
model.add(Embedding(vocab_size, output_dim=EMBEDDING_SIZE, input_length=NUM_STEPS, embeddings_initializer=initializer))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True, kernel_initializer=initializer, recurrent_initializer=initializer, bias_initializer=initializer))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax', kernel_initializer=initializer, bias_initializer=initializer)))
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 8, 64)             640000    
_________________________________________________________________
lstm_21 (LSTM)               (None, 8, 64)             33024     
_________________________________________________________________
time_distributed_20 (TimeDis (None, 8, 10000)          650000    
Total params: 1,323,024
Trainable params: 1,323,024
Non-trainable params: 0
_________________________________________________________________


In [55]:
for i in range(5):
    generator = keras_generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)
    model.fit_generator(
        generator,
        steps_per_epoch=((len(X_train) // BATCH_SIZE - 1)// NUM_STEPS),
    )
    val_gen = keras_generate_epoch(X_val, BATCH_SIZE, NUM_STEPS)
    val_loss = model.evaluate_generator(
        val_gen,
        steps=((len(X_val) // BATCH_SIZE - 1)// NUM_STEPS)
    )
    print('val perplexity: {0:.1f}'.format(np.exp(val_loss)))
#    K.backend.set_value(model.optimizer.lr, 0.333 * K.backend.get_value(model.optimizer.lr))

Epoch 1/1
1815/1815 [==============================] - 26s 14ms/step - loss: 5.6798
val perplexity: 239.3
Epoch 1/1
1815/1815 [==============================] - 25s 14ms/step - loss: 5.2110
val perplexity: 219.3
Epoch 1/1
1815/1815 [==============================] - 26s 14ms/step - loss: 5.0573
val perplexity: 216.5
Epoch 1/1
 297/1815 [===>..........................] - ETA: 21s - loss: 4.9939

KeyboardInterrupt: 

In [31]:
K.backend.set_value(model.optimizer.lr, 1e-3)

In [70]:
x.shape

(128, 8)

In [71]:
y.shape

(128, 8)

In [72]:
y_pred = model.predict(x)

ValueError: Error when checking : expected embedding_19_input to have shape (None, 16) but got array with shape (128, 8)

In [73]:
my_gen = keras_generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)

In [74]:
x, y = next(my_gen)

In [75]:
y_pred = model.predict(x)

In [76]:
y_pred.shape

(32, 16, 10000)

In [77]:
y_pred[0,0]

array([  1.0005e-04,   9.9916e-05,   1.0011e-04, ...,   9.9993e-05,   1.0000e-04,   1.0004e-04], dtype=float32)

In [78]:
np.sum(y_pred[0,0])

1.0